### Simple genAI app using openai

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGSMITH_ENDPOINT"]=os.getenv("LANGSMITH_ENDPOINT")

In [ ]:
## Data ingestion -- we need to scrape the data from the website
from langchain_community.document_loaders import WebBaseLoader

In [7]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/billing#optimize-your-tracing-spend")
loader 

In [8]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing#optimize-your-tracing-spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='Manage billing in your account - Docs by LangChainOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationSetupOverviewCreate an account and API keySet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesFAQsTroubleshootingCloud architecture and scalabilityRegions FAQAuthentication methodsData purging for complianceRelease versionsOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KAsk AIGitHubForumForumSearch...NavigationSetupManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringSelf

In [9]:
### Load data --> docs --> divide the docs into chunks --> vector --> vector embeddings --> store in a vector store db
from langchain_text_splitters import RecursiveCharacterTextSplitter
textsplitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = textsplitter.split_documents(docs)

In [11]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing#optimize-your-tracing-spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='Manage billing in your account - Docs by LangChainOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationSetupOverviewCreate an account and API keySet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesFAQsTroubleshootingCloud architecture and scalabilityRegions FAQAuthentication methodsData purging for complianceRelease versionsOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KAsk AIGitHubForumForumSearch...NavigationSetupManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringSelf

In [12]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [13]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [14]:
vectorstoredb

In [17]:
query = "update your organization’s business information"
result = vectorstoredb.similarity_search(query)
print(result[0].page_content)

To update your organization’s business information, follow these steps:

Navigate to the Plans and Billing tab.
Below the invoice email section, you will find a checkbox labeled Business.
Check the Business checkbox if your organization belongs to a business.
A business information section will appear, allowing you to enter or update the following details:

Business Name
Address
Tax ID for applicable jurisdictions


A Tax ID field will appear for applicable jurisdictions after you select a country.
After entering the necessary information, click the Save button to save your changes.

This ensures that your business information is up-to-date and accurate for billing and tax purposes.
​Optimize your tracing spend
You may find it helpful to read the following pages, before continuing with this section on optimizing your tracing spend:
Data Retention Conceptual Docs
Usage Limiting Conceptual Docs


### Retrieval chain, Document chain

In [18]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """ 
        Answer the following questions based on provided context:
        <context>
        {context}
        </context> 
    """
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' \n        Answer the following questions based on provided context:\n        <context>\n        {context}\n        </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f00d00356d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f00d0035a90>, root_client=<openai.OpenAI object at 0x7f00d0034910>, root_async_client=<openai.AsyncOpenAI object at 0x7f00d0035810>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs=

In [22]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"update your organization’s business information",
    "context":[Document(page_content="update your organization’s business information, follow these steps: Navigate to the Plans and Billing tab.")]
})

"Sure, feel free to ask your questions related to the provided context about updating your organization's business information via the Plans and Billing tab."

## Retriever

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [23]:
#### input --> retriever --> vectorstoredb
vectorstoredb

In [28]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [29]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f00f6963b60>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' \n        Answer the following questions based on provided context:\n        <context>\n        {context}\n        </context>\n    '), additional_kwargs

In [ ]:
## get the response from the llm
response = retrieval_chain.invoke({"input":"Check the Business checkbox"})
response['answer'].

'1. How can you update your organization\'s business information if you belong to a business?\n\nTo update your organization\'s business information, navigate to the Plans and Billing tab. Below the invoice email section, check the checkbox labeled "Business" if your organization belongs to a business. A business information section will then appear where you can enter or update details like Business Name, Address, and Tax ID for applicable jurisdictions. Make sure to click the Save button after entering the necessary information.\n\n2. Can you update business information and tax IDs for the Free and Developer plans?\n\nNo, business information, tax ID, and invoice email updates can only be made for the Plus and Startup plans. The Free and Developer plans do not allow updates to this information.\n\n3. How can you qualify for a sales tax exemption?\n\nTo qualify for a sales tax exemption, if you are a business, you need to provide your tax ID in the business information section. This m

In [31]:
response

{'input': 'Check the Business checkbox',
 'context': [Document(id='62852bc8-bcc1-44ef-8349-255697f9ab3c', metadata={'source': 'https://docs.langchain.com/langsmith/billing#optimize-your-tracing-spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='To update your organization’s business information, follow these steps:\n\nNavigate to the Plans and Billing tab.\nBelow the invoice email section, you will find a checkbox labeled Business.\nCheck the Business checkbox if your organization belongs to a business.\nA business information section will appear, allowing you to enter or update the following details:\n\nBusiness Name\nAddress\nTax ID for applicable jurisdictions\n\n\nA Tax ID field will appear for applicable jurisdictions after you select a country.\nAfter entering the necessary information, click the Save button to save your changes.\n\nThis ensures that your business information is up-to-date and accurate for billing and tax purpo

In [32]:
response['context']

[Document(id='62852bc8-bcc1-44ef-8349-255697f9ab3c', metadata={'source': 'https://docs.langchain.com/langsmith/billing#optimize-your-tracing-spend', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='To update your organization’s business information, follow these steps:\n\nNavigate to the Plans and Billing tab.\nBelow the invoice email section, you will find a checkbox labeled Business.\nCheck the Business checkbox if your organization belongs to a business.\nA business information section will appear, allowing you to enter or update the following details:\n\nBusiness Name\nAddress\nTax ID for applicable jurisdictions\n\n\nA Tax ID field will appear for applicable jurisdictions after you select a country.\nAfter entering the necessary information, click the Save button to save your changes.\n\nThis ensures that your business information is up-to-date and accurate for billing and tax purposes.\n\u200bOptimize your tracing spend\nYou may find